In [1]:
import pandas as pd

## Preparing MM_SCORE data 

In [2]:
mm_df = pd.read_csv('data/mm_score.csv', sep='\t')

In [3]:
mm_df = mm_df.drop(['MM_REQ_ID', 'Trainer_OK', 'Update_Date'], axis=1)
mm_df.columns = map(str.lower, mm_df.columns)

In [4]:
mm_df = mm_df.pivot_table(values='req_level', index='mode_id', columns='sill_id').reset_index()

In [5]:
parametrs = mm_df.drop(['mode_id'], axis=1)
names_list = parametrs.columns

In [6]:
mm_df['param'] = parametrs.apply(lambda row: [[row[x] for x in names_list]], axis=1)

In [7]:
mm_df = mm_df[['mode_id', 'param']]

In [8]:
mm_df['param'] = mm_df['param'].apply(lambda x: tuple(x[0]))

## Preparing STAFF_SCORE data

In [9]:
staff_df = pd.read_csv('data/staff_score.csv', sep='\t')

In [10]:
staff_df = staff_df.drop(['Score_ID', 'Trainer_OK', 'Update_Date'], axis=1)
staff_df.columns = map(str.lower, staff_df.columns)

In [11]:
staff_df.columns

Index(['empl_id', 'skill_id', 'score_value'], dtype='object')

In [12]:
staff_df = staff_df.pivot_table(values='score_value', index='empl_id', columns='skill_id').reset_index()

In [13]:
parametrs = staff_df.drop(['empl_id'], axis=1)
names_list = parametrs.columns

In [14]:
staff_df['param'] = parametrs.apply(lambda row: [[row[x] for x in names_list]], axis=1)

In [15]:
staff_df = staff_df[['empl_id', 'param']]
staff_df['param'] = staff_df['param'].apply(lambda x: tuple(x[0]))

In [16]:
staff_df['key'] = 0
mm_df['key'] = 0
df = staff_df.merge(mm_df, on='key')
df.drop(labels='key', axis=1, inplace=True)
df.head()

,empl_id,param_x,mode_id,param_y
0,105,"(2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 3, 0, ...",1_1_1_MM,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
1,105,"(2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 3, 0, ...",1_1_2_MM,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
2,105,"(2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 3, 0, ...",1_2_1_MM,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,105,"(2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 3, 0, ...",1_2_2_MM,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, ..."
4,105,"(2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 3, 0, ...",1_3_1_MM,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, ..."


In [17]:
df['resource'] = df['param_x'].apply(lambda x: sum(x))
df['needs'] = df['param_y'].apply(lambda x: sum(x))

In [18]:
df['subtraction'] = df[['param_x', 'param_y']].apply(lambda row: [x - y for x, y in zip(row['param_x'], row['param_y'])], axis=1)

In [19]:
df['coef_+'] = df['subtraction'].apply(lambda x: sum([i for i in x if i > 0]))
df['coef_-'] = df['subtraction'].apply(lambda x: sum([i for i in x if i < 0]))

In [20]:
df['absolute_error'] = df['subtraction'].apply(lambda x: sum([abs(i) for i in x]))

In [21]:
sum(df['resource'])

55020

In [22]:
df['test_coeficient'] = df[['needs', 'resource', 'absolute_error']].apply(
    lambda x: (df['resource'].sum() - x['resource']) / (df['needs'].sum() - x['needs']) / x['absolute_error'], axis=1)

In [23]:
df = df.sort_values(by='test_coeficient', ascending=False)

In [24]:
res_list = [df.iloc[0].values]
row = df.iloc[0]

In [25]:
for i in df.values:
    if i[0] not in [x[0] for x in res_list] and i[3] not in [x[3] for x in res_list]:
        res_list.append(i)
        row = i
    else:
        pass

In [26]:
res_df = pd.DataFrame(res_list, columns=df.columns)

In [27]:
sum(res_df['coef_-'])

-232

In [28]:
sum(res_df['coef_+'])

1309

In [29]:
staff = pd.read_csv('data/staff.csv', sep='\t', header=None, names=['empl_id', 's_name'])

In [30]:
res_df

,empl_id,param_x,mode_id,param_y,resource,needs,subtraction,coef_+,coef_-,absolute_error,test_coeficient
0,41150,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5_1_1_MM,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",0,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0,...",0,-5,5,0.632305
1,46454,"(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1_1_1_MM,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",11,6,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0,...",10,-5,15,0.210738
2,41148,"(2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2_1_1_MM,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",13,6,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0,...",11,-4,15,0.210731
3,81834,"(2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4_1_1_MM,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",16,6,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0,...",14,-4,18,0.175599
4,41123,"(2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1_1_2_MM,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",16,9,"[2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0,...",13,-6,19,0.166386
5,57679,"(2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5_1_2_MM,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",22,9,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0,...",17,-4,21,0.150523
6,61414,"(2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",2_1_2_MM,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",46,9,"[2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",37,0,37,0.085395
7,81740,"(2, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 1, 0, ...",3_1_2_MM,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",50,9,"[2, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, ...",42,-1,43,0.073474
8,45469,"(2, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 1, 0, ...",4_1_2_MM,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",49,9,"[2, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, ...",42,-2,44,0.071805
9,69219,"(2, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 1, 0, ...",3_1_1_MM,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",50,6,"[2, 1, 1, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, ...",44,0,44,0.071792


In [31]:
res_df = res_df[['empl_id', 'mode_id', 'resource', 'needs', 'coef_+', 'coef_-', 'absolute_error', 'test_coeficient']].join(staff, on='empl_id', lsuffix='_a', rsuffix='_b')

In [32]:
res_df.drop('empl_id_b', axis=1, inplace=True)

In [33]:
res_df.rename(columns={'empl_id_a': 'empl_id'}, inplace=True)

In [34]:
cols = res_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
res_df = res_df[cols]

In [35]:
res_df.to_excel('result_test_1.xlsx', index=False)

In [391]:
mach = pd.read_csv('data/machine.csv', sep='\t')
mach.columns = map(str.lower, mach.columns)

In [392]:
mach.drop(['machine_mode'], axis=1, inplace=True)

In [400]:
res_df = res_df.join(mach, lsuffix='_a', rsuffix='_b', how='left')

In [401]:
cols = res_df.columns.tolist()
cols = cols[-2:] + cols[:-2]
res_df = res_df[cols]

In [404]:
res_df.drop('mode_id_b', axis=1, inplace=True)

In [407]:
res_df.sort_values(by='mode_id_a', inplace=True)

In [409]:
res_df.to_excel('res_version_1.xlsx', index=False)

In [412]:
staff_df['resource'] = staff_df['param'].apply(lambda x: sum(x))
mm_df['needs'] = mm_df['param'].apply(lambda x: sum(x))

In [416]:
sum(staff_df['resource'])

1834

In [417]:
sum(mm_df['needs'])

544

In [418]:
res_df

,machine_name,machine_sub_mode,s_name,empl_id,mode_id_a,resource,needs,coef_+,coef_-,absolute_error,test_coeficient
1,SASL5AD,With Adjustments,Мамчич,46454,1_1_1_MM,11,6,10,-5,15,0.210738
4,SASL5AD,New Item Start-Up,Ковальчук,41123,1_1_2_MM,16,9,13,-6,19,0.166386
10,3344АЕ,New Item Start-Up,Бугайчук,42235,1_2_1_MM,49,14,40,-5,45,0.070230
15,3П772-2,Item Change,Шафранюк,81606,1_2_2_MM,49,22,40,-13,53,0.059657
18,CL660,Normal Operating,Григоренко,9995,1_3_1_MM,49,27,40,-18,58,0.054529
26,MTD250,Abrasive or Diamond Change,Гордовський,10034,1_3_2_MM,94,31,70,-7,77,0.041050
2,SASL5AD,Abrasive or Diamond Change,Коханець,41148,2_1_1_MM,13,6,11,-4,15,0.210731
6,3344АЕ,Normal Operating,Скрябін,61414,2_1_2_MM,46,9,37,0,37,0.085395
11,3344АЕ,New Technology Implementation,Белоусова,3143,2_2_1_MM,50,14,41,-5,46,0.068702
16,3П772-2,New Item Start-Up,Місірук,45470,2_2_2_MM,49,22,42,-15,57,0.055470
